# Spectral Trust Demo

This notebook demonstrates how to use the `spectral_trust` package to analyze a simple sentence.

In [ ]:
from spectral_trust import GSPDiagnosticsFramework, GSPConfig
import torch

# Check if CUDA is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
# Initialize configuration
model_name = "meta-llama/Llama-3.2-1B"  # Full HuggingFace Model ID
config = GSPConfig(
    model_name=model_name,
    device=device,
    save_plots=True,
    output_dir="./demo_results"
)

In [ ]:
# Run analysis
text = "The capital of France is Paris."

with GSPDiagnosticsFramework(config) as framework:
    framework.instrumenter.load_model(config.model_name)
    results = framework.analyze_text(text)
    
    print(f"Analyzed: {text}")
    print("\nFirst 5 layers energy:")
    for diag in results['layer_diagnostics'][:5]:
        print(f"Layer {diag.layer}: {diag.energy:.4f}")